# Initial Data Cleaning of Semmed DB

There are two main problems found when first looking into semmedDB

1. There are several rows where there are multiple subjects or objects, sepearted by a pipe `|` character.
2. The database is not entirely in CUI space, some concepts are given entrez gene ids.

There is a third minor problem of data corruption, however this is on less than 0.001% of the data, so when identified, these will just be removed.

In [1]:
import os
import pickle
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
sem_df = pd.read_csv('../data/semmedVER31_R.csv')

/home/mmayers/software/anaconda3/envs/ml/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print('Rows: {:,}'.format(sem_df.shape[0]))
print('Cols: {}'.format(sem_df.shape[1]))

Rows: 93,876,632
Cols: 12


In [4]:
sem_df.head()

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
0,38999,87169,4958141,PART_OF,C0040291,Tissue Extracts,bacs,1,C0085979,Cavia,mamm,1
1,39000,87167,13997680,LOCATION_OF,C0005767,Blood,tisu,1,C0007061,Carboxyhemoglobin,aapp,1
2,39001,87171,12254865,PROCESS_OF,C0702166,Acne,dsyn,1,C0043210,Woman,humn,1
3,39002,87175,14847741,PART_OF,C0027809,Neurilemmoma,neop,1,C0038351,Stomach,bpoc,1
4,39003,87165,11396507,PART_OF,C0221921,Stratum corneum,tisu,1,C0020114,Human,humn,1


In [5]:
# Get all the pmids and save them to a file
pmids = set(sem_df['PMID'])
out = []
print('{:,}'.format(len(pmids)))
for pmid in pmids:
    try:
        out.append(int(pmid))
    except:
        pass
print('{:,}'.format(len(out)))
with open('../data/pmid_list_ver31.txt', 'w') as out_file:
    for pmid in out:
        out_file.write(str(pmid)+'\n')
print('Done!')

17,472,087
17,471,829
Done!


# Expanding the pipes in subjects and objects

One of the first things that was noticed upon look at the data in semmedDB was that some subjects and objects of extracted statments contained the pipe character `|` as an indicator of multiple concepts in the sentence.

## Examining Pipes in Subject/Object IDs

First thing to do is just examine some of these pipes and took at their correspodning sentences in the database, see if they do infact correspond to two concepts.

In [6]:
multi_subject = sem_df[sem_df['SUBJECT_CUI'].str.contains('|', regex=False)]
print("There are {:,} lines that contain a pipe in the subject".format(multi_subject.shape[0]))

sentence_ids = multi_subject['SENTENCE_ID'].values

There are 3,551,682 lines that contain a pipe in the subject


In [7]:
multi_subject.iloc[:10]

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
213,39212,87854,13384314,INTERACTS_WITH,C1366490|156,ADRBK1 gene|ADRBK1,gngm,1,C0040615,Antipsychotic Agents,phsu,1
290,39289,88096,14782737,ASSOCIATED_WITH,C0001655|5443,Corticotropin|POMC,horm,1,C0021311,Infection,dsyn,0
354,39353,88296,4229045,PART_OF,C0032140|5340,Plasminogen|PLG,aapp,1,C0020114,Human,humn,1
785,39784,89525,18933908,PART_OF,C0001924|213,Albumins|ALB,aapp,1,C0020114,Human,humn,1
818,39817,89640,6065080,PART_OF,C0001457|100,ADENOSINE DEAMINASE|ADA,aapp,1,C0013303,Duodenum,bpoc,1
1045,40044,90238,4165881,PART_OF,C0034833|5241,"Receptors, Progesterone|PGR",aapp,1,C0027567,African race,humn,0
1050,40049,90257,13680884,PART_OF,C0017796|2744,Glutaminase|GLS,aapp,1,C0014792,Erythrocytes,cell,1
1122,40121,90444,14770469,TREATS,C0001655|5443,Corticotropin|POMC,aapp,1,C0023418,leukemia,neop,1
1236,40235,90801,14880803,TREATS,C0001655|5443,Corticotropin|POMC,horm,1,C0003873,Rheumatoid Arthritis,dsyn,1
1743,40742,92151,13680882,PART_OF,C0001457|100,ADENOSINE DEAMINASE|ADA,aapp,1,C0014792,Erythrocytes,cell,1


Before we go any further, lets drop any rows that conatin NaN values, as these are corrupted rows that have no good data

In [8]:
# Remove any NaN values
print('Rows before NaN removal {:,}'.format(sem_df.shape[0]))
sem_df = sem_df.dropna()
print('Rows after NaN removal {:,}'.format(sem_df.shape[0]))

Rows before NaN removal 93,876,632
Rows after NaN removal 93,876,632


In [9]:
multi_start = sem_df['SUBJECT_CUI'].str.contains('|', regex=False)
multi_end = sem_df['OBJECT_CUI'].str.contains('|', regex=False)

In [10]:
pipe_lines = sem_df[multi_start | multi_end]
good_lines = sem_df[~multi_start & ~multi_end]
print('Rows with multiple subjects or objects {:,}'.format(len(pipe_lines)))
print('Rows with only 1 subject AND only 1 object {:,}'.format(len(good_lines)))

Rows with multiple subjects or objects 6,316,090
Rows with only 1 subject AND only 1 object 87,560,542


Lines with a pipe in the subject XOR a pipe in the object can be delt with in a rather straightforward mannor. 

Those with a pipe in both the subject AND the object will require a slightly different algorithm, so we'll separate those out.

In [11]:
# get indices for those only with a multi start, multi end, and those with bith a multi start and multi end
multi_start_subset = multi_start[multi_start | multi_end]
multi_end_subset = multi_end[multi_start | multi_end]
multi_both_subset = multi_start_subset & multi_end_subset

In [12]:
start_only_subset = multi_start_subset & ~multi_end_subset
end_only_subset = multi_end_subset & ~multi_start_subset

### Splitting the IDs of the Subjects OR Objects

To split the IDs, the IDs and names will be split into `n+1` rows where `n` is the number of pipes `|`, then the data from the rest of the columns will be duplicated across these new rows.

In [13]:
from itertools import chain

In [14]:
# Split the IDs and Names
start_id_split = pipe_lines.loc[start_only_subset, 'SUBJECT_CUI'].str.split('|')
start_name_split = pipe_lines.loc[start_only_subset, 'SUBJECT_NAME'].str.split('|')

In [15]:
# Get the number of items after splitting
start_lens = start_id_split.apply(len)

In [16]:
# Need the column names for duplicating the data
all_cols = list(pipe_lines.columns)

# Copy the columns and only keep those where the data will be duped
start_cols = all_cols[:]
start_cols.remove('SUBJECT_CUI')
start_cols.remove('SUBJECT_NAME')

In [17]:
# Retaining the same order, duplicate the data times of the new number of rows after the split
new_starts = dict()
for c in start_cols:
    tmp = pipe_lines.loc[start_only_subset, c].apply(lambda x: [x]) * start_lens
    new_starts[c] = [x for x in chain(*tmp.values)]

In [18]:
# Now we have the expanded rows with everthing except the subject CUIs and Names
fixed_starts = pd.DataFrame(new_starts)
fixed_starts.head(10)

,OBJECT_CUI,OBJECT_NAME,OBJECT_NOVELTY,OBJECT_SEMTYPE,PMID,PREDICATE,PREDICATION_ID,SENTENCE_ID,SUBJECT_NOVELTY,SUBJECT_SEMTYPE
0,C0040615,Antipsychotic Agents,1,phsu,13384314,INTERACTS_WITH,39212,87854,1,gngm
1,C0040615,Antipsychotic Agents,1,phsu,13384314,INTERACTS_WITH,39212,87854,1,gngm
2,C0021311,Infection,0,dsyn,14782737,ASSOCIATED_WITH,39289,88096,1,horm
3,C0021311,Infection,0,dsyn,14782737,ASSOCIATED_WITH,39289,88096,1,horm
4,C0020114,Human,1,humn,4229045,PART_OF,39353,88296,1,aapp
5,C0020114,Human,1,humn,4229045,PART_OF,39353,88296,1,aapp
6,C0020114,Human,1,humn,18933908,PART_OF,39784,89525,1,aapp
7,C0020114,Human,1,humn,18933908,PART_OF,39784,89525,1,aapp
8,C0013303,Duodenum,1,bpoc,6065080,PART_OF,39817,89640,1,aapp
9,C0013303,Duodenum,1,bpoc,6065080,PART_OF,39817,89640,1,aapp


In [19]:
# Add in the subject CUIs and Names
fixed_starts['SUBJECT_CUI'] = [x for x in chain(*start_id_split.values)]
fixed_starts['SUBJECT_NAME'] = [x for x in chain(*start_name_split.values)]

fixed_starts = fixed_starts[all_cols]

In [20]:
fixed_starts.head(10)

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
0,39212,87854,13384314,INTERACTS_WITH,C1366490,ADRBK1 gene,gngm,1,C0040615,Antipsychotic Agents,phsu,1
1,39212,87854,13384314,INTERACTS_WITH,156,ADRBK1,gngm,1,C0040615,Antipsychotic Agents,phsu,1
2,39289,88096,14782737,ASSOCIATED_WITH,C0001655,Corticotropin,horm,1,C0021311,Infection,dsyn,0
3,39289,88096,14782737,ASSOCIATED_WITH,5443,POMC,horm,1,C0021311,Infection,dsyn,0
4,39353,88296,4229045,PART_OF,C0032140,Plasminogen,aapp,1,C0020114,Human,humn,1
5,39353,88296,4229045,PART_OF,5340,PLG,aapp,1,C0020114,Human,humn,1
6,39784,89525,18933908,PART_OF,C0001924,Albumins,aapp,1,C0020114,Human,humn,1
7,39784,89525,18933908,PART_OF,213,ALB,aapp,1,C0020114,Human,humn,1
8,39817,89640,6065080,PART_OF,C0001457,ADENOSINE DEAMINASE,aapp,1,C0013303,Duodenum,bpoc,1
9,39817,89640,6065080,PART_OF,100,ADA,aapp,1,C0013303,Duodenum,bpoc,1


#### Fixing the lines where the Objects contain pipes

In [21]:
end_id_split = pipe_lines.loc[end_only_subset, 'OBJECT_CUI'].str.split('|')
end_name_split = pipe_lines.loc[end_only_subset, 'OBJECT_NAME'].str.split('|')

When examining the data, we can see that some of the lines were not parsed correctly.  This must have happened before the data was download, because mysql shows the same issues when the dump is loaded and queried. 

These line will be dropped since there aren't many and they're pretty much garbage.

In [22]:
end_lens = end_id_split.apply(len)
end_lens1 = end_name_split.apply(len)

print('There are {} lines with data corruped in this manner'.format(sum(end_lens != end_lens1)))

pipe_lines.loc[end_only_subset][(end_lens != end_lens1)]

There are 0 lines with data corruped in this manner


,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY


In [23]:
# get the index for the bad lines
bad_lines = pipe_lines.loc[end_only_subset][(end_lens != end_lens1)].index

# Remove them from the main dataframe
pipe_lines = pipe_lines.drop(bad_lines)

# Remove from the indicies that still need to be used as well...
end_only_subset = end_only_subset.drop(bad_lines)
multi_both_subset = multi_both_subset.drop(bad_lines)

Now the splitting algorithm is identical to that used for the Subject lines

In [24]:
end_id_split = pipe_lines.loc[end_only_subset, 'OBJECT_CUI'].str.split('|')
end_name_split = pipe_lines.loc[end_only_subset, 'OBJECT_NAME'].str.split('|')
end_lens = end_id_split.apply(len)

In [25]:
end_cols = all_cols[:]
end_cols.remove('OBJECT_CUI')
end_cols.remove('OBJECT_NAME')

In [26]:
new_ends = dict()
for c in end_cols:
    tmp = pipe_lines.loc[end_only_subset, c].apply(lambda x: [x]) * end_lens
    new_ends[c] = [x for x in chain(*tmp.values)]

In [27]:
fixed_ends = pd.DataFrame(new_ends)
fixed_ends['OBJECT_CUI'] = [x for x in chain(*end_id_split.values)]
fixed_ends['OBJECT_NAME'] = [x for x in chain(*end_name_split.values)]

fixed_ends = fixed_ends[all_cols]

In [28]:
fixed_ends.head()

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
0,39356,88174,4291298,LOCATION_OF,C0006104,Brain,bpoc,1,C0001655,Corticotropin,aapp,1
1,39356,88174,4291298,LOCATION_OF,C0006104,Brain,bpoc,1,5443,POMC,aapp,1
2,39870,89730,14770466,USES,C0087111,Therapeutic procedure,topp,0,C0001655,Corticotropin,aapp,1
3,39870,89730,14770466,USES,C0087111,Therapeutic procedure,topp,0,5443,POMC,aapp,1
4,40015,90197,13738601,LOCATION_OF,C0022646,Kidney,bpoc,1,708,C1QBP,aapp,1


### Splitting of lines where both the subject and object contain pipes

These differ slightly in the way that they will have to be treated.  First, the number of pipe in the subject and object can be different.  The total number of new lines to be created is `(n+1) * (m+1)` where `n` is the number of pipes in the subject and `m` is the number of pipes in the object.

Secondly, every possible combination of subject and object will need to be made.  Given subjects `A` and `B`, and objects `X` and `Y`, and predicate `p` you will need rows for the following combinatinos `ApX`, `ApY`, `BpX`, `BpY`.

In [29]:
start_id_split = pipe_lines.loc[multi_both_subset, 'SUBJECT_CUI'].str.split('|')
start_name_split = pipe_lines.loc[multi_both_subset, 'SUBJECT_NAME'].str.split('|')
start_lens = start_id_split.apply(len)

end_id_split = pipe_lines.loc[multi_both_subset, 'OBJECT_CUI'].str.split('|')
end_name_split = pipe_lines.loc[multi_both_subset, 'OBJECT_NAME'].str.split('|')
end_lens = end_id_split.apply(len)

In [30]:
# Multiply the start splits by the end length, so you get end_len*start_len total rows
start_id_split = start_id_split * end_lens
start_name_split = start_name_split * end_lens

end_id_split = end_id_split * start_lens
end_name_split = end_name_split * start_lens

In [31]:
# only sort the starts so that you get all possible combinations....
# For example right now we have start = [A, B, C, A, B, C] and end = [X, Y, X, Y, X, Y]
# By sorting the start we will have start = [A, A, B, B, C, C] and end = [X, Y, X, Y, X, Y]
# Therefore when combined element-wise, all possible combinatinos will arise

sorting_df = pd.DataFrame()
sorting_df['ID'] = start_id_split
sorting_df['NAME'] = start_name_split

sorted_start_id_split = sorting_df['ID'].apply(lambda x: sorted(x))
# Need to sort the names based on IDs so that the same name still corresponds to the same ID
sorted_start_name_split = sorting_df.apply(lambda row: [x for y,x in sorted(zip(row['ID'], row['NAME']))], axis = 1)

In [32]:
sorting_df.head()

,ID,NAME
45788,"[C0001924, 213, C0001924, 213, C0001924, 213]","[Albumins, ALB, Albumins, ALB, Albumins, ALB]"
47644,"[C0001655, 5443, C0001655, 5443]","[Corticotropin, POMC, Corticotropin, POMC]"
51488,"[C0755813, 325, 4068, C0755813, 325, 4068, C07...","[SKAP55-related protein, APCS, SH2D1A, SKAP55-..."
51517,"[C0755813, 325, 4068, C0755813, 325, 4068, C07...","[SKAP55-related protein, APCS, SH2D1A, SKAP55-..."
51548,"[C0755813, 325, 4068, C0755813, 325, 4068]","[SKAP55-related protein, APCS, SH2D1A, SKAP55-..."


In [33]:
sorted_start_id_split.head()

45788        [213, 213, 213, C0001924, C0001924, C0001924]
47644                     [5443, 5443, C0001655, C0001655]
51488    [325, 325, 325, 4068, 4068, 4068, C0755813, C0...
51517    [325, 325, 325, 4068, 4068, 4068, C0755813, C0...
51548           [325, 325, 4068, 4068, C0755813, C0755813]
Name: ID, dtype: object

In [34]:
sorted_start_name_split.head()

45788        [ALB, ALB, ALB, Albumins, Albumins, Albumins]
47644           [POMC, POMC, Corticotropin, Corticotropin]
51488    [APCS, APCS, APCS, SH2D1A, SH2D1A, SH2D1A, SKA...
51517    [APCS, APCS, APCS, SH2D1A, SH2D1A, SH2D1A, SKA...
51548    [APCS, APCS, SH2D1A, SH2D1A, SKAP55-related pr...
dtype: object

Now the algorithm continues in a simialr mannor to that of the Only subject or Only Object corrections

In [35]:
both_cols = all_cols[:]
both_cols.remove('SUBJECT_CUI')
both_cols.remove('SUBJECT_NAME')
both_cols.remove('OBJECT_CUI')
both_cols.remove('OBJECT_NAME')

In [36]:
new_both = dict()
for c in both_cols:
    tmp = pipe_lines.loc[multi_both_subset, c].apply(lambda x: [x]) * (start_lens * end_lens)
    new_both[c] = [x for x in chain(*tmp.values)]

In [37]:
fixed_both = pd.DataFrame(new_both)

fixed_both['SUBJECT_CUI'] = [x for x in chain(*sorted_start_id_split.values)]
fixed_both['SUBJECT_NAME'] = [x for x in chain(*sorted_start_name_split.values)]

fixed_both['OBJECT_CUI'] = [x for x in chain(*end_id_split.values)]
fixed_both['OBJECT_NAME'] = [x for x in chain(*end_name_split.values)]

fixed_both = fixed_both[all_cols]

In [38]:
fixed_both.head()

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
0,84787,211961,5634523,PART_OF,213,ALB,bacs,1,C1412332,ALB gene,aapp,1
1,84787,211961,5634523,PART_OF,213,ALB,bacs,1,213,ALB,aapp,1
2,84787,211961,5634523,PART_OF,213,ALB,bacs,1,85302,FBF1,aapp,1
3,84787,211961,5634523,PART_OF,C0001924,Albumins,bacs,1,C1412332,ALB gene,aapp,1
4,84787,211961,5634523,PART_OF,C0001924,Albumins,bacs,1,213,ALB,aapp,1


Recombine these lines into the new dataframe.

In [39]:
sem_df = pd.concat([good_lines, fixed_starts, fixed_ends, fixed_both]).reset_index(drop=True)

In [40]:
print('The data now contains {:,} rows'.format(sem_df.shape[0]))

The data now contains 102,230,673 rows


# NORMALIZE IDS FOR GENES to CUIs

Sometimes genes appear with a CUI as an identifier, other times they have an entrez gene id.

In [41]:
# POMC Gene is 5443 and has CUI C1337111
sem_df.query('SUBJECT_CUI == "C1337111"').head()

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
87560644,48911,115433,13029321,TREATS,C1337111,POMC gene,aapp,1,C0023449,"Leukemia, Lymphocytic, Acute",neop,1
87561498,123811,320122,13584735,NEG_AFFECTS,C1337111,POMC gene,gngm,1,C0232804,Renal function,ortf,1
87568264,760451,2104994,13954719,PREVENTS,C1337111,POMC gene,aapp,1,C0023418,leukemia,neop,1
87568278,763776,2114043,13176585,ISA,C1337111,POMC gene,aapp,1,C0087111,Therapeutic procedure,topp,0
87568280,763805,2114043,13176585,TREATS,C1337111,POMC gene,aapp,1,C0023467,"Leukemia, Myelocytic, Acute",neop,1


In [42]:
sem_df.query('SUBJECT_CUI == "5443"').head()

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
589719,633940,1759696,13841870,INHIBITS,5443,POMC,aapp,1,C0001655,Corticotropin,aapp,1
2707630,2792160,6541453,4366303,INTERACTS_WITH,5443,POMC,aapp,1,C0001655,Corticotropin,aapp,1
2869332,2957401,6883310,203610,NEG_AUGMENTS,5443,POMC,gngm,1,C0871828,Neophobia,menp,1
2939862,3029653,7029365,195674,PART_OF,5443,POMC,gngm,1,C0034693,Rattus norvegicus,mamm,1
2940091,3029887,7029770,195674,COEXISTS_WITH,5443,POMC,gngm,1,C0030956,Peptides,aapp,1


## Strategy for normalizing to CUI

Finding a direct map from enterz to CUI is not straightforward. The UMLS API has a way to map from HGNC ID to cui, however, not directly from Entrez ID (or at least I couldn't figure it out).  

We will be mapping to HGNC id first, then to CUI.

In [43]:
# Map Created from WIKI DATA see README.txt for more info
e_to_h = pd.read_csv('../entrez_to_hgnc.csv')

In [44]:
e_to_h = e_to_h.set_index('entrez_id')['hgnc_id'].to_dict()

In [45]:
gene_lines = ~sem_df['SUBJECT_CUI'].str.startswith('C')
genes_entrez = set(sem_df.loc[gene_lines, 'SUBJECT_CUI'])

gene_lines1 = ~sem_df['OBJECT_CUI'].str.startswith('C')
genes_entrez.update(set(sem_df.loc[gene_lines1, 'OBJECT_CUI']))

genes_need_fixing = gene_lines | gene_lines1

In [46]:
print("{} genes appear with Entrez IDs that will need to be mapped".format(len(genes_entrez)))

19270 genes appear with Entrez IDs that will need to be mapped


### Generate Enterez to CUI map

In [47]:
import requests
from tqdm import tqdm
from pyquery import PyQuery as pq

In order to use the UMLS api (which is pretty ridiculously limited in its capabilities)
An api key is needed.  This can be obtained by creating an account with them https://uts.nlm.nih.gov//license.html
then logging into the account and going to the 'My Profile' page.

In [48]:
with open('../data/api.key', 'r') as fin: 
    TICKET = fin.read().rstrip()

In [49]:
def handshake():
    # insane umls api instructions
    # https://documentation.uts.nlm.nih.gov/rest/authentication.html
    r = requests.post("https://utslogin.nlm.nih.gov/cas/v1/api-key", data={'apikey': TICKET})
    d = pq(r.text)
    tgt = d.find('form').attr('action')
    return tgt

In [50]:
def hgnc_to_umls(hgnc_id, tgt):
    """
    given an hgnc_id, get the umls cui
    """
    data = {'service': 'http://umlsks.nlm.nih.gov'}
    r = requests.post(tgt, data=data)
    st = r.text
    url = "https://uts-ws.nlm.nih.gov/rest/content/current/source/HGNC/HGNC:{}/atoms?ticket={}"
    d = requests.get(url.format(hgnc_id, st)).json()
    for res in d['result']:
        if res['code'].split('/')[-1] == 'HGNC:{}'.format(hgnc_id) and 'concept' in res.keys():
            return res['concept'].split('/')[-1]
    
    print('no match')
    return float('NaN')

In [51]:
def cui_to_name(cui, tgt):
    """
    given a name, get the umls cui
    """
    data = {'service': 'http://umlsks.nlm.nih.gov'}
    r = requests.post(tgt, data=data)
    st = r.text
    url = "https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{}?ticket={}"
    d = requests.get(url.format(cui, st)).json()
    return d['result']['name']

In [52]:
def get_entrez_to_cui(ids, tgt):
    mapper = dict()
    for eid in tqdm(ids):
        try:
            hid = e_to_h.get(int(eid), None)
            if not hid:
                continue
            else:
                cui = hgnc_to_umls(hid, tgt)
                mapper[eid] = cui
        except:
            continue

    return mapper

def get_cui_to_name(cuis, tgt):
    mapper = dict()
    for cui in tqdm(cuis):
        name = cui_to_name(cui, tgt)
        mapper[cui] = name
    return mapper

In [53]:
tgt = handshake()

In [54]:
# Because their API takes so long to run (handshake process, no easy way to query many IDs at once)
# We will save the results to a pickle, so future runs don't need to be repeated
if os.path.exists("../data/entrez_to_cui.pkl"):
    e_to_cui = pickle.load(open( "../data/entrez_to_cui.pkl", "rb" ))
else:
    e_to_cui = dict()

to_query = set(genes_entrez) - set(e_to_cui.keys())

In [55]:
print("{} of the original {} still do not have a map and will be queried.".format(len(to_query), len(genes_entrez)))

560 of the original 19270 still do not have a map and will be queried.


In [56]:
# The typical response rate is about 1 query per second
# Thats about 5 horus for the whole data

# However if the ID cannot be found, the next query is isntant
# So the 568 which seem to not be mappable query very quickly
if to_query:
    query_result = get_entrez_to_cui(to_query, tgt)
    e_to_cui.update(query_result)

100%|██████████| 560/560 [00:04<00:00, 122.65it/s]


In [57]:
got_cuis = set(e_to_cui.keys())

no_cuis = genes_entrez - got_cuis

print("Unable to map {} of the original {} enterz genes.".format(len(no_cuis), len(genes_entrez)))

Unable to map 560 of the original 19270 enterz genes.


In [58]:
pickle.dump(e_to_cui, open( "../data/entrez_to_cui.pkl", "wb" ) )

In [59]:
# Qucick test to see if all the cuis we quieried for are in-fact CUIs
total = 0
for cui in e_to_cui.values():
    if not cui.startswith('C'):
        total += 1
print(total)  # should be 0

0


### Generate a CUI to name map


Once the Entrez IDs are changed to CUIs, the names will not be the true name that is associated with that CUI.  We will use the UMLS API to get the correct names. This will ensure the correct UMLS name appears with the CUI.


In [60]:
# First get the names from semmed for everything that already has a CUI
d = sem_df[sem_df['SUBJECT_CUI'].str.startswith('C')].set_index('SUBJECT_CUI')['SUBJECT_NAME'].to_dict()
od = sem_df[sem_df['OBJECT_CUI'].str.startswith('C')].set_index('OBJECT_CUI')['OBJECT_NAME'].to_dict()

c_to_name_dict = {**d, **od}

In [61]:
len(c_to_name_dict)

260144

In [62]:
if os.path.exists("../data/cui_to_name.pkl"):
    c_to_name_dict.update(pickle.load(open( "../data/cui_to_name.pkl", "rb" )))

to_query = set(e_to_cui.values()) - set(c_to_name_dict.keys())

In [63]:
len(to_query)

0

In [64]:
if to_query:
    tgt = handshake()
    query_result = get_cui_to_name(to_query, tgt)
    c_to_name_dict.update(query_result)

In [65]:
len(c_to_name_dict)

271620

In [66]:
pickle.dump(c_to_name_dict, open( "../data/cui_to_name.pkl", "wb" ) )

In [67]:
# Check that the mapper produces the correct name when given the CUI for POMC gene
c_to_name_dict[e_to_cui['5443']]

'POMC gene'

### Apply the Changes

In [68]:
genes_need_fixing = gene_lines | gene_lines1

sem_df[genes_need_fixing].head(10)

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
2815,41841,95436,4892304,PART_OF,820,CAMP,gngm,1,C0949876,Nazis,humn,1
3344,42376,96958,20342456,LOCATION_OF,C0003062,Animals,anim,0,55578,SUPT20H,aapp,1
5126,44178,102317,13930654,PART_OF,C0036825,Serum Proteins,bacs,1,567,B2M,aapp,1
5205,44257,102517,13738792,LOCATION_OF,C0459385,Brain tissue,bpoc,1,23038,WDTC1,aapp,1
6278,45345,105754,13310143,PART_OF,115825,WDFY2,gngm,1,C0402112,Scientist,humn,0
6486,45556,106326,4872999,AFFECTS,1652,DDT,gngm,1,C0018270,Growth,orgf,1
7581,46657,109086,4221862,PART_OF,23038,WDTC1,gngm,1,C0034652,Rana esculenta,amph,1
7616,46692,109086,4221862,PART_OF,51761,ATP8A2,gngm,1,C0034652,Rana esculenta,amph,1
7660,46736,109086,4221862,PART_OF,51761,ATP8A2,gngm,1,C0260307,Triturus cristatus,amph,1
7700,46776,109086,4221862,PART_OF,23038,WDTC1,gngm,1,C0260307,Triturus cristatus,amph,1


In [69]:
sem_df.loc[genes_need_fixing, 'SUBJECT_CUI'] = sem_df.loc[genes_need_fixing, 'SUBJECT_CUI'].apply(lambda e: e_to_cui.get(e,e))
sem_df.loc[genes_need_fixing, 'OBJECT_CUI'] = sem_df.loc[genes_need_fixing, 'OBJECT_CUI'].apply(lambda e: e_to_cui.get(e,e))

In [70]:
sem_df.loc[genes_need_fixing, 'SUBJECT_NAME'] = sem_df.loc[genes_need_fixing, 'SUBJECT_CUI'].apply(lambda e: c_to_name_dict.get(e,e))
sem_df.loc[genes_need_fixing, 'OBJECT_NAME'] = sem_df.loc[genes_need_fixing, 'OBJECT_CUI'].apply(lambda e: c_to_name_dict.get(e,e))

In [71]:
sem_df[genes_need_fixing].head(10)

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
2815,41841,95436,4892304,PART_OF,C1413106,CAMP gene,gngm,1,C0949876,Nazis,humn,1
3344,42376,96958,20342456,LOCATION_OF,C0003062,Animals,anim,0,C1539449,SUPT20H gene,aapp,1
5126,44178,102317,13930654,PART_OF,C0036825,Serum Proteins,bacs,1,C1412709,B2M gene,aapp,1
5205,44257,102517,13738792,LOCATION_OF,C0459385,Brain tissue,bpoc,1,C1428806,WDTC1 gene,aapp,1
6278,45345,105754,13310143,PART_OF,C1426966,WDFY2 gene,gngm,1,C0402112,Scientist,humn,0
6486,45556,106326,4872999,AFFECTS,C1413950,DDT gene,gngm,1,C0018270,Growth,orgf,1
7581,46657,109086,4221862,PART_OF,C1428806,WDTC1 gene,gngm,1,C0034652,Rana esculenta,amph,1
7616,46692,109086,4221862,PART_OF,C1366628,ATP8A2 gene,gngm,1,C0034652,Rana esculenta,amph,1
7660,46736,109086,4221862,PART_OF,C1366628,ATP8A2 gene,gngm,1,C0260307,Triturus cristatus,amph,1
7700,46776,109086,4221862,PART_OF,C1428806,WDTC1 gene,gngm,1,C0260307,Triturus cristatus,amph,1


### The lines that got no Query Result

Some of the ID produced no query result.  They should still have a SUBJECT_CUI with only a number. We'll examine those and see if they produce any insight.

In [72]:
gene_lines2 = ~sem_df['SUBJECT_CUI'].str.startswith('C')
gene_lines3 = ~sem_df['OBJECT_CUI'].str.startswith('C')

genes_need_fixing1 = gene_lines2 | gene_lines3

sem_df[genes_need_fixing1]

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
54563,94072,236814,5699417,AFFECTS,8066,8066,gngm,1,C0234222,Baresthesia,ortf,1
78984,118725,305854,13032397,USES,C0033573,Prostatectomy,topp,1,100188776,100188776,aapp,1
144137,184485,486340,13388624,PART_OF,C0870883,Metabolites,bacs,1,619511,619511,aapp,1
146503,186864,493343,4961715,PART_OF,57306,57306,gngm,1,C0162547,Pseudomonas Phages,virs,1
286382,327941,894450,5637040,DISRUPTS,100188784,100188784,gngm,1,C1328948,RNA Synthesis,moft,1
288201,329778,899989,5217476,PART_OF,C0035668,RNA,bacs,1,100271694,100271694,aapp,1
315546,357320,980127,5723528,PART_OF,474256,474256,gngm,1,C0004651,Bacteriophages,virs,1
327550,369417,1015074,14192648,PART_OF,C0035668,RNA,bacs,1,100271694,100271694,aapp,1
406805,449380,1248536,5638139,PART_OF,544326,544326,gngm,1,C0007125,"Carcinoma, Ehrlich Tumor",neop,1
421409,464105,1291919,14114111,ASSOCIATED_WITH,100188864,100188864,gngm,1,C0027708,Nephroblastoma,neop,1


For now we will save these to their own file and remove them from the 'cleaned' data.

In [73]:
sem_df[genes_need_fixing1].to_csv('../data/semmedVER31_R_no_CUI.csv', index=False)

In [74]:
sem_df = sem_df.drop(sem_df[genes_need_fixing1].index)
sem_df.to_csv('../data/semmedVER31_R_clean.csv', index=False)

## Remove Depricated CUIs

Some CUIs in the database are depreicated.  They may have newer versions to which they have not yet been mapped.  However, UMLS has record of these deprecated values that can be used to map 

In [75]:
# Get the map from old CUIs to new CUIs
cols = ['CUI1', 'VER', 'REL', 'RELA', 'MAPREASON', 'CUI2', 'MAPIN', 'extra']
retired_cui = pd.read_csv('../2017AA-full/2017AA/META/MRCUI.RRF', sep='|', header=None, names=cols)
retired_cui.head()

/home/mmayers/software/anaconda3/envs/ml/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CUI1,VER,REL,RELA,MAPREASON,CUI2,MAPIN,extra
0,C0000002,2000AC,SY,NaN,NaN,C0007404,Y,NaN
1,C0000003,1999AA,SY,NaN,NaN,C0010504,Y,NaN
2,C0000024,1993AA,SY,NaN,NaN,C0043791,Y,NaN
3,C0000105,1995AA,SY,NaN,NaN,C0001964,Y,NaN
4,C0000136,1993AA,DEL,NaN,NaN,NaN,NaN,NaN


In [76]:
# Make a mapper from the old to the new
cui_map = retired_cui.set_index('CUI1')['CUI2'].to_dict()

# Ensure we have names for all the new values
no_name = set(cui_map.values()) - set(c_to_name_dict.keys())
if len(no_name) > 0:
    tgt = handshake()
    query_result = get_cui_to_name(to_query, tgt)
    c_to_name_dict.update(query_result)
    pickle.dump(c_to_name_dict, open( "../data/cui_to_name.pkl", "wb" ) )

0it [00:00, ?it/s]


In [77]:
# How many unique spo triples before de-depreication?
len(sem_df.drop_duplicates(subset=['SUBJECT_CUI', 'PREDICATE', 'OBJECT_CUI']))

21229373

In [78]:
# Map the depricated values to their new CUIs
sem_df['SUBJECT_CUI'] = sem_df['SUBJECT_CUI'].apply(lambda c: cui_map.get(c, c))
sem_df['OBJECT_CUI'] = sem_df['OBJECT_CUI'].apply(lambda c: cui_map.get(c, c))

# Any removed CUIs should be taken out
sem_df = sem_df.dropna(subset=['SUBJECT_CUI', 'OBJECT_CUI'])

# Ensure the names are now corrected
sem_df['SUBJECT_NAME'] = sem_df['SUBJECT_CUI'].apply(lambda c: c_to_name_dict.get(c, c))
sem_df['OBJECT_NAME'] = sem_df['OBJECT_CUI'].apply(lambda c: c_to_name_dict.get(c, c))

# How many unique spo triples after the corrections?
len(sem_df.drop_duplicates(subset=['SUBJECT_CUI', 'PREDICATE', 'OBJECT_CUI']))

21029688

In [79]:
sem_df.to_csv('../data/semmedVER31_R_clean_de-depricate.csv', index=False)